# Ex ? Manipulating Spectra Part 2 - `Spectra`

## Notebook Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("../..")

In [2]:
%matplotlib inline

In [3]:
%autoreload


import matplotlib
matplotlib.rc_file('../matplotlibrc')
import matplotlib.pyplot as plt

import pickle

import numpy as np
from scipy.integrate import quad
from scipy.interpolate import interp2d

import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
import darkhistory.spec.pppc as pppc
import darkhistory.history.tla as tla
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra

import main
import config

In [ ]:
rs_vec = np.flipud(np.arange(1., 3000., 0.1))
soln_default_reion = tla.get_history(rs_vec, helium_TLA=True, reion_switch=True, mxstep=0)
soln_default_reion_2 = tla.get_history(rs_vec, helium_TLA=True, reion_switch=True)

In [4]:
%autoreload
soln_DM_reion_BR = main.evolve(
    DM_process='decay', mDM=1e9, lifetime=3e25, primary='elec_delta',
    start_rs = 3000., end_rs=2900.,
    coarsen_factor=1, backreaction=True, helium_TLA=True, reion_switch=True
)

****** Loading transfer functions... ******
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******


In [ ]:
%autoreload
helium_method_alt = main.evolve(
     DM_process='decay', mDM=1e8,
     lifetime=3e24, primary='phot_delta',
     start_rs=3000., end_rs = 5., backreaction=True,
     helium_TLA=True, coarsen_factor=4,
     compute_fs_method='He_recomb'
)

3000.0 -0.3503127499343267 0.9999999992990679 0.07885956786310544 9.999986949615902e-13
2999.985913611738 -0.3468081327013215 0.9999999992990095 0.07887142895855966 9.999986949615902e-13
2999.985913611738 -0.4143334675731064 0.9999999992990095 0.07887147409637321 9.999986949615902e-13
2999.9964784029344 -0.3494365956260754 0.9999999992990534 0.07886253880711486 9.999986949615902e-13
2999.9964784029344 -0.35366248626989927 0.9999999992990534 0.0788625416336996 9.999986949615902e-13
2999.9991196007336 -0.35009371135726386 0.9999999992990642 0.07886031095402227 9.999986949615902e-13
2999.9991196007336 -0.3503579164485907 0.9999999992990642 0.07886031113076916 9.999986949615902e-13
2999.9991196007336 -0.3500396506197222 0.9999999992990642 0.07886031112388486 9.999986949615902e-13
2999.9997799001835 -0.350257990290061 0.999999999299067 0.0788597536580251 9.999986949615902e-13
2999.9997799001835 -0.3502745044668589 0.999999999299067 0.0788597536690731 9.999986949615902e-13
2999.999559800367 

2988.0239680319733 -0.3541312650041489 0.9999999992912321 0.08102340592801408 9.999986949615902e-13
2982.059516100807 -0.35133263396103975 0.9999999992912321 0.08101649364793134 9.999986949615902e-13
2982.059516100807 -39.13894166676901 0.9999999992912321 0.08102574660167686 9.999986949615902e-13
2986.5328550491818 -0.3534316072433716 0.9999999992912321 0.08102171989241888 9.999986949615902e-13
2986.5328550491818 -2.785947367586896 0.9999999992912321 0.08102236609815389 9.999986949615902e-13
2987.651189786275 -0.3539563505639546 0.9999999992912321 0.08102298699316501 9.999986949615902e-13
2987.651189786275 -0.5061183300877808 0.9999999992912321 0.08102302821035748 9.999986949615902e-13
2987.930773470549 -0.3540875363941003 0.9999999992912321 0.08102330135436336 9.999986949615902e-13
2987.930773470549 -0.36359968819430843 0.9999999992912321 0.08102330394214147 9.999986949615902e-13
2988.000669391617 -0.35412033285163674 0.9999999992912321 0.08102337979459254 9.999986949615902e-13
2988.0

2954.3915614284324 -0.36543870092851743 0.9999999992912321 0.08097625833377389 9.999986949615902e-13
2954.3915614284324 -0.3654387063739785 0.9999999992912323 0.08097625833377389 9.999986949615902e-13
2954.3915614284324 -0.3654387063739785 0.9999999992912321 0.08097625834777618 9.999986949615902e-13
2954.3915614284324 -0.3654387063739785 0.9999999992912321 0.08097625833377389 9.999986949615902e-13
2954.3915614284324 -0.3654502423822619 0.9999999992912321 0.08097615329229275 9.999986949615902e-13
2964.2151385857933 -0.36213204320118164 0.9999999992912321 0.08099194527927928 9.999986949615902e-13
2961.8105761604124 -0.358521172104835 0.9999999992912321 0.08098834155598553 9.999986949615902e-13
2961.8105761604124 -14.652165043786216 0.9999999992912321 0.0809920609478732 9.999986949615902e-13
2963.613997979448 -0.36122932542709496 0.9999999992912321 0.08099105328725828 9.999986949615902e-13
2963.613997979448 -1.256715733473348 0.9999999992912321 0.080991293673488 9.999986949615902e-13
2964

2948.5943027374724 -0.36740958457209105 0.9999999992912321 0.08096596591411903 9.999986949615902e-13
2948.5943027374724 -0.3674144085235754 0.9999999992912321 0.08096592847567735 9.999986949615902e-13
2952.3819601658583 -0.3661320928198361 0.9999999992912321 0.08097267608922619 9.999986949615902e-13
2950.0330033694013 -0.36248089300629566 0.9999999992912321 0.08096860773660743 9.999986949615902e-13
2950.0330033694013 -14.331629641048634 0.9999999992912321 0.08097204232599566 9.999986949615902e-13
2951.794720966744 -0.365219292866451 0.9999999992912321 0.0809716689596852 9.999986949615902e-13
2951.794720966744 -1.2403737832061148 0.9999999992912321 0.08097189022655557 9.999986949615902e-13
2952.23515036608 -0.36590389283148983 0.9999999992912321 0.08097242492419578 9.999986949615902e-13
2952.23515036608 -0.4206336412697919 0.9999999992912321 0.0809724388320936 9.999986949615902e-13
2952.3452577159137 -0.36607504282274955 0.9999999992912321 0.08097261333647472 9.999986949615902e-13
2952.

2940.59601992027 -0.37013168201414137 0.9999999992912321 0.08095058830881978 9.999986949615902e-13
2936.892815433768 -0.3664583544283193 0.9999999992912321 0.08094305524085035 9.999986949615902e-13
2936.892815433768 -24.651550026012195 0.9999999992912321 0.08094870863746804 9.999986949615902e-13
2939.6702187986443 -0.36921335011768586 0.9999999992912321 0.08094873459624213 9.999986949615902e-13
2939.6702187986443 -1.8915113773484888 0.9999999992912321 0.0809491050912999 9.999986949615902e-13
2940.3645696398635 -0.36990209904002747 0.9999999992912321 0.08095012670378038 9.999986949615902e-13
2940.3645696398635 -0.4651158440454986 0.9999999992912321 0.08095015007344863 9.999986949615902e-13
2940.5381573501686 -0.3700742862706129 0.9999999992912321 0.08095047302113209 9.999986949615902e-13
2940.5381573501686 -0.3760262414070617 0.9999999992912321 0.08095047448482827 9.999986949615902e-13
2940.5815542777445 -0.3701173330782593 0.9999999992912321 0.08095055949399033 9.999986949615902e-13
29

...

246.25499587176398 -2.889843290758333 0.0009834899091030747 1.8835116144429176e-05 9.999986949615902e-13
245.2719433041487 -2.894074607566425 0.0009852411095271907 1.8892426546776586e-05 9.999986949615902e-13
245.2719433041487 -2.8940909467547233 0.0009852845460786575 1.889304050294721e-05 9.999986949615902e-13
245.2719433041487 -2.8940895219114076 0.0009852839502311195 1.8893034001515874e-05 9.999986949615902e-13
244.29281509285872 -2.89832189528432 0.000987040505906367 1.895021560065907e-05 9.999986949615902e-13
244.29281509285872 -2.8983382052594786 0.000987083981198067 1.8950830712523292e-05 9.999986949615902e-13
244.29281509285872 -2.8983367929206714 0.0009870833853649064 1.8950824212465855e-05 9.999986949615902e-13
243.3175955718218 -2.9025702281170385 0.0009888451769331508 1.9007874773678546e-05 9.999986949615902e-13
243.3175955718218 -2.9025865087019556 0.0009888886916881723 1.9008491029426366e-05 9.999986949615902e-13
243.3175955718218 -2.902585108764888 0.0009888880958666135 

220.1628649399275 -3.00911079982546 0.001035847274741375 2.0417694733446123e-05 9.999986949615902e-13
219.28397243703003 -3.013370285986428 0.0010377482033198104 2.047219911449627e-05 9.999986949615902e-13
219.28397243703003 -3.0133857634895516 0.001037792831575457 2.0472842271322678e-05 9.999986949615902e-13
219.28397243703003 -3.0133846513101394 0.0010377922350398028 2.0472835800353084e-05 9.999986949615902e-13
218.40858848236962 -3.017645177054503 0.0010396973421968325 2.052722984019484e-05 9.999986949615902e-13
218.40858848236962 -3.017660619899485 0.0010397420206356145 2.052787392915062e-05 9.999986949615902e-13
218.40858848236962 -3.017659517697761 0.0010397414240547187 2.0527867460116977e-05 9.999986949615902e-13
217.53669906978433 -3.0219210866504547 0.0010416506049679697 2.0582148991865612e-05 9.999986949615902e-13
217.53669906978433 -3.0219364945765173 0.001041695334104309 2.0582794003042026e-05 9.999986949615902e-13
217.53669906978433 -3.0219354022762603 0.001041694737478504

197.62426327926792 -3.1248437577890695 0.001089800298334831 2.1867955433345354e-05 9.999986949615902e-13
196.83534511437134 -3.1291290632615407 0.0010918111747622028 2.191954634202936e-05 9.999986949615902e-13
196.83534511437134 -3.1291435476671046 0.001091857216848291 2.192021155784235e-05 9.999986949615902e-13
196.83534511437134 -3.12914267806403 0.0010918566191220336 2.192020515760784e-05 9.999986949615902e-13
196.04957631919575 -3.1334288957045446 0.0010938710112710082 2.1971681250475272e-05 9.999986949615902e-13
196.04957631919575 -3.133443338814294 0.0010939171093817812 2.1972347166297757e-05 9.999986949615902e-13
196.04957631919575 -3.1334424773119767 0.0010939165116251592 2.1972340769671472e-05 9.999986949615902e-13
195.26694432142364 -3.1377296090024434 0.0010959343239746455 2.2023700027622162e-05 9.999986949615902e-13
195.26694432142364 -3.1377440104094516 0.0010959804784880789 2.2024366633371928e-05 9.999986949615902e-13
195.26694432142364 -3.1377431569527805 0.0010959798807

177.39299244370682 -3.241212437141011 0.001146404076466212 2.32350211138016e-05 9.999986949615902e-13
176.6848377275704 -3.2455199676580864 0.0011484989561820202 2.3283318373833184e-05 9.999986949615902e-13
176.6848377275704 -3.2455332295334705 0.0011485465585122645 2.3283999243763065e-05 9.999986949615902e-13
176.6848377275704 -3.245532558756238 0.0011485459608859827 2.328399297054501e-05 9.999986949615902e-13
175.97950997260688 -3.2498407504646023 0.0011506439988496209 2.3332174197782107e-05 9.999986949615902e-13
175.97950997260688 -3.249853962180089 0.0011506916595075412 2.3332855546252806e-05 9.999986949615902e-13
175.97950997260688 -3.24985329804354 0.0011506910619189514 2.333284927843321e-05 9.999986949615902e-13
175.27699789355714 -3.2541621494646833 0.0011527921749611703 2.338091279923543e-05 9.999986949615902e-13
175.27699789355714 -3.254175310484273 0.0011528398942171614 2.3381594616678214e-05 9.999986949615902e-13
175.27699789355714 -3.2541746529471793 0.001152839296670427 2

158.5971862151023 -3.362397952678938 0.001207331810165524 2.4556833787867105e-05 9.999986949615902e-13
157.96406455771873 -3.3667214373886463 0.00120949376710483 2.4601423622053198e-05 9.999986949615902e-13
157.96406455771873 -3.366733121463688 0.001209543053344031 2.4602114096453076e-05 9.999986949615902e-13
157.96406455771873 -3.3667326205760806 0.0012095424585137904 2.4602108016552604e-05 9.999986949615902e-13
157.33347032873797 -3.3710563594078007 0.0012117073343599905 2.4646580745496527e-05 9.999986949615902e-13
157.33347032873797 -3.371067981582327 0.0012117566791964851 2.4647271481950672e-05 9.999986949615902e-13
157.33347032873797 -3.3710674861393586 0.001211756084513238 2.4647265409440122e-05 9.999986949615902e-13
156.7053934386389 -3.3753914717080016 0.001213923815562623 2.4691619982725177e-05 9.999986949615902e-13
156.7053934386389 -3.3754030313423793 0.0012139732191646102 2.4692310973219167e-05 9.999986949615902e-13
156.7053934386389 -3.3754025413120274 0.001213972624633463

In [18]:
# 6.5166964471677336 -5.40010279095005 0.0036154037267939754 3.27730025470388e-05 9.999986949615902e-13
%autoreload
phys.C_He(0.0036, 3.277e-5, 6.51, 'triplet')
# phys.beta_ion(phys.TCMB(6.51), 'HeI_23s')

1.0

In [ ]:
%autoreload
rs_vec = np.flipud(np.arange(1., 3000., 0.1))
soln_He = tla.get_history(rs_vec, helium_TLA=True, init_cond=np.array([phys.TCMB(3000.), 1, phys.chi, 0]))

In [ ]:
plt.figure()

plt.loglog()

# plt.plot(rs_vec, soln_default_reion[:,0])
# plt.plot(rs_vec, soln_default_reion_2[:,0])
plt.plot(soln_DM_reion_BR['rs'], soln_DM_reion_BR['x'][:,1])

In [ ]:
config.load_data('dep_tf')

In [ ]:
import copy

# a = copy.deepcopy(config.glob_dep_tf_data['highengphot'])

for i,grid in enumerate(a.grid_vals):
    a.grid_vals[i] = grid.astype(np.float32)
    print(a.grid_vals[i].dtype)

In [ ]:
for grid in config.glob_dep_tf_data['lowengphot'].grid_vals:
    grid = grid.astype('float32')

In [ ]:
for grid in config.glob_dep_tf_data['lowengelec'].grid_vals:
    grid = grid.astype('float32')

In [ ]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size


def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
pickle_dump(
    a,'/Users/hongwan/Dropbox (MIT)/Photon Deposition/DarkHistory_data/highengphot_tf_interp_flt32.raw'
)
# pickle.dump(
#     config.glob_dep_tf_data['lowengphot'], 
#     open('/Users/hongwan/Dropbox (MIT)/Photon Deposition/DarkHistory_data/lowengphot_tf_interp_flt32.raw','wb')
# )
# pickle.dump(
#     config.glob_dep_tf_data['lowengelec'], 
#     open('/Users/hongwan/Dropbox (MIT)/Photon Deposition/DarkHistory_data/lowengelec_tf_interp_flt32.raw','wb')
# )

In [ ]:
photeng = 10**np.arange(-4, 12, 0.05)
a = pppc.get_pppc_spec(100e9, photeng, 'gamma', 'phot')
b = pppc.get_pppc_spec(100e9, photeng, 'gamma', 'elec')

In [ ]:
plt.figure()
plt.loglog()

plt.plot(photeng, a.dNdE)

In [ ]:
a.toteng()/200e9 + b.toteng()/200e9

In [ ]:
f_elec_CMB_raw = np.loadtxt(config.data_path+'/fFromIonBaseline3keV_electron_extended.dat', delimiter=',')
log10eng_elec_CMB  = f_elec_CMB_raw[0:2760:69, 0]
log10rs_elec_CMB = f_elec_CMB_raw[0:69, 1] 

f_phot_CMB_raw = np.loadtxt(config.data_path+'/fFromIonBaseline3keV_photon.dat', delimiter=',')
log10eng_phot_CMB  = f_phot_CMB_raw[0:2800:70, 0]
log10rs_phot_CMB = f_phot_CMB_raw[0:70, 1] 

f_elec_CMB_raw = np.transpose(np.reshape(f_elec_CMB_raw[:,2], (40,69)))
f_phot_CMB_raw = np.transpose(np.reshape(f_phot_CMB_raw[:,2], (40,70)))

f_elec_CMB = interp2d(log10eng_elec_CMB, log10rs_elec_CMB, f_elec_CMB_raw)
f_phot_CMB = interp2d(log10eng_phot_CMB, log10rs_phot_CMB, f_phot_CMB_raw)

def xsec_bound_phot_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return 4.1e-28*(mDM*1e-9)/f_phot_CMB(np.log10(mDM), np.log10(601))[0]


In [ ]:
max_xsec = xsec_bound_phot_CMB(1e6, 'swave')

In [ ]:
# result_no_BR = main.evolve(
#             primary='phot_delta',
#             DM_process='decay', mDM=1e7, lifetime = 3e24,
#             start_rs = 3000, end_rs=17,
#             coarsen_factor=4, backreaction=False
#         )

mDM_arr = 10**np.arange(4.01, 6.26, 0.25)

# result_no_BR = [ 
#     main.evolve(
#         primary='phot_delta',
#         DM_process='swave', mDM=mDM, sigmav = xsec_bound_phot_CMB(mDM, 'swave'),
#         start_rs = 3000, end_rs=4.1,
#         coarsen_factor=4, backreaction=False, 
#         struct_boost = phys.struct_boost_func()
#     ) for mDM in mDM_arr
# ]

result_BR = [
    main.evolve(
        primary='phot_delta',
        DM_process='swave', mDM=mDM, sigmav = xsec_bound_phot_CMB(mDM, 'swave'),
        start_rs = 3000, end_rs=4.1,
        coarsen_factor=4, backreaction=True, 
        struct_boost = phys.struct_boost_func()
    ) for mDM in mDM_arr
]

In [ ]:
Tm_no_BR = np.array([res['Tm'] for res in result_no_BR])
Tm_BR    = np.array([res['Tm'] for res in result_BR])

In [ ]:
import darkhistory.utilities as utils

utils.compare_arr([Tm_no_BR[0], Tm_BR[0]])

In [ ]:
plt.figure()

plt.loglog()

frac_change_levels = [1e-5, .1, .5, 1, 2, 5, 10, 20, 40]

plt.contour(
    mDM_arr, result_no_BR[0]['rs'], np.transpose((Tm_BR/Tm_no_BR - 1.)*100),
    frac_change_levels,
    norm=matplotlib.colors.LogNorm(),
    cmap=matplotlib.cm.inferno
)

plt.axis([1e4, 5e12, 4.1, 200])

In [ ]:
test = main.evolve(
        primary='elec_delta',
        DM_process='swave', mDM=1e6, sigmav = xsec_bound_phot_CMB(1e6, 'swave'),
        start_rs = 3000, end_rs=4.004,
        coarsen_factor=12, backreaction=True, 
        struct_boost = phys.struct_boost_func()
) 

In [ ]:
test['rs'].size

In [ ]:
utils.compare_arr([3000*np.exp(-.012*np.arange(552)) , test['rs']])

In [ ]:
plt.figure()

# plt.loglog()

plt.plot(result_no_BR['rs'], result_BR['Tm']/result_no_BR['Tm'] - 1)

plt.axis([1, 20, 0, 0.5])

In [ ]:
inj_str = 'elec'
inj_type = 'swave'
extra = ''

f_DH_list_new = pickle.load(open('/Users/hongwan/Dropbox (MIT)/Photon Deposition/idl_comparison/GR_Checks/ih_continuous_'
                   +inj_str+'/f_'+inj_str+'_'+inj_type+'_notCrossCheck_LowPHigh.p','rb')
                  )

In [ ]:
print(f_DH_list_new.shape)
print(eng_arr.shape)
print(rs_vec.shape)


In [ ]:
rs_vec = 3000*np.exp(-.001*np.arange(6620))
eng_arr = 5e3*10**(np.arange(40)*0.23252559)
eng_arr[-1]=10**12.60150599

plt.figure()

plt.loglog()

# plt.contour(eng_arr, rs_vec, np.transpose(np.log10(f_DH_list[:,:,0])), levels=np.log10([0.01, 0.1, 0.2, 0.5, 1, 5, 10, 50]))
# plt.contour(eng_arr, rs_vec, np.transpose(np.log10(f_elec_swave_heat)), levels=np.log10(np.array([0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 5, 10])))

plt.contour(
    eng_arr, rs_vec, np.transpose(np.log10(f_DH_list_new[:,0,:])), 
    levels=np.log10(np.array([10])), 
#     colors='k', linestyles='dashed'
)



In [ ]:
f_phot_swave_heat = np.array(f_phot_swave_heat)
print(f_phot_swave_heat.shape)

In [ ]:
from tqdm import tqdm_notebook as tqdm

%autoreload
f_phot_swave_heat = [
    [
        phys.f_std(eng, rs, inj_particle='phot', inj_type='swave', channel='heat', struct=True)
        for rs in rs_vec
    ]
    for eng in tqdm(eng_arr)
]


 

In [ ]:
import main

result = main.evolve(
            primary='elec_delta',
            DM_process='swave', mDM=eng_arr[10] + phys.me, sigmav = 3e-26,
            start_rs = 3000, end_rs=5.1,
            coarsen_factor=12, backreaction=False,
            struct_boost = phys.struct_boost_func('einasto_with_subs')
        )

In [ ]:
result['f']['high']

In [ ]:
plt.figure()

plt.loglog()

plt.plot(result['rs'], result['f']['low']['H ion'] + result['f']['high']['H ion'], 'k--')
plt.plot(rs_vec, f_DH_list_new[10,0,:])
plt.axis([4, 4000, 10**(-3.5), 1000])

In [ ]:
eng_arr[10]

In [ ]:
eng = np.array([1, 10, 100, 1000])
spec_arr = [Spectrum(eng, np.ones(4)*i, rs=100, spec_type='N') for i in np.arange(4)]
test_spectra = Spectra(spec_arr)
test_spectra.redshift(np.array([0.01, 0.1, 1, 10]))
print(test_spectra.grid_vals)

In [ ]:
a = Spectrum(eng, np.ones(4)*3, rs=100, spec_type='N') 
a.totN(bound_type='bin')

In [ ]:
0.5*13*10 + 14*100 + 15*1000

In [ ]:
9*0.5 + 10 + 11

In [ ]:
12*1 + 13*10 + 14*100 + 15*1000

In [ ]:
1*10 + 2*100

In [ ]:
%autoreload 2
phys.Tm_std(100)

In [ ]:
a = phys.struct_boost_func()

In [ ]:
a(1)

In [ ]:
%autoreload
phys.f_std(3e5, 300, inj_particle='phot', inj_type='decay', channel='heat')

In [ ]:
from config import load_data

In [ ]:
a = load_data('dep_tf')

In [ ]:
hep_tf = a['highengphot']
lep_tf  = a['lowengphot']
lee_tf  = a['lowengelec']

In [ ]:
%%cython

def coarsen_matrices(hep, lep, lee, cf):
    prop_tf = np.zeros_like(hep)
    for i in np.arange(cf):
        prop_tf += np.linalg.matrix_power(hep, i)
    lep = np.matmul(prop_tf, lep)
    lee = np.matmul(prop_tf, lee)
    hep = np.linalg.matrix_power(hep, cf)



In [ ]:
from tqdm import tqdm_notebook as tqdm

cf = 12

for (hep_reg, lep_reg, lee_reg) in zip(hep_tf.grid_vals, lep_tf.grid_vals, lee_tf.grid_vals):
    for (hep_dim_1, lep_dim_1, lee_dim_1) in zip(tqdm(hep_reg), lep_reg, lee_reg):
        if hep_dim_1.ndim == 2:
            coarsen_matrices(hep_dim_1, lep_dim_1, lee_dim_1, cf)
        else:
            for (hep_dim_2, lep_dim_2, lee_dim_2) in zip(tqdm(hep_dim_1), lep_dim_1, lee_dim_1):
                if hep_dim_2.ndim == 2:
                    coarsen_matrices(hep_dim_2, lep_dim_2, lee_dim_2, cf)
                else:
                    for (hep_dim_3, lep_dim_3, lee_dim_3) in zip(hep_dim_2, lep_dim_2, lee_dim_2):
                        coarsen_matrices(hep_dim_3, lep_dim_3, lee_dim_3, cf)
        
            